In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from functools import partial

In [2]:
key_file = open("/Users/tristanbrigham/GithubProjects/AI_Training_Data/LicensePlateProject/training_data.txt", "r")
alphabet = open("/Users/tristanbrigham/GithubProjects/AI_Training_Data/LicensePlateProject/alphabet.txt")
image_path = "/Users/tristanbrigham/GithubProjects/AI_Training_Data/LicensePlateProject/"

key_lines = key_file.readlines()
max_val = int(key_lines[0])
print("The max value is: {}".format(max_val))

The max value is: 3055


In [3]:
key_dict = {}
return_letter_dict = {}
for count, letter in enumerate(alphabet.readlines()):
    key_dict[letter[0]] = count
    return_letter_dict[count] = letter[0]

print(key_dict)
print(return_letter_dict)

{'_': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26, '0': 27, '1': 28, '2': 29, '3': 30, '4': 31, '5': 32, '6': 33, '7': 34, '8': 35, '9': 36}
{0: '_', 1: 'A', 2: 'B', 3: 'C', 4: 'D', 5: 'E', 6: 'F', 7: 'G', 8: 'H', 9: 'I', 10: 'J', 11: 'K', 12: 'L', 13: 'M', 14: 'N', 15: 'O', 16: 'P', 17: 'Q', 18: 'R', 19: 'S', 20: 'T', 21: 'U', 22: 'V', 23: 'W', 24: 'X', 25: 'Y', 26: 'Z', 27: '0', 28: '1', 29: '2', 30: '3', 31: '4', 32: '5', 33: '6', 34: '7', 35: '8', 36: '9'}


In [4]:
keys = []
letters = []
for number in range(1, max_val):
    letter_image = np.loadtxt(image_path + str(number) + ".txt").reshape(80, 60) / 255.0
    letters.append(letter_image)
    
    array = np.zeros(37, dtype="float32")
    key_char = key_lines[number][0]
    array[key_dict[key_char]] = float(1.0)
    keys.append(np.argmax(array))
    
    if number % 500 == 0:
        print("Completed {}".format(number))

Completed 500
Completed 1000
Completed 1500
Completed 2000
Completed 2500
Completed 3000


In [5]:
letters[0].shape

(80, 60)

In [6]:
DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=3, activation='relu', padding="SAME")

model = keras.models.Sequential([
    keras.layers.AveragePooling2D(pool_size=2, input_shape=[80, 60, 1]),
    keras.layers.Conv2D(filters=64, kernel_size=13, name="First_Conv2D", padding="SAME", activation='relu'),
    keras.layers.AveragePooling2D(pool_size=2),
    DefaultConv2D(filters=128, kernel_size=9, name="Second_Conv2D"),
    keras.layers.AveragePooling2D(pool_size=2),
    
    keras.layers.Flatten(),
    
#     keras.layers.Dense(units = 8, activation='relu', name="Dense_1"),
#     keras.layers.Dense(units = 2400, activation='relu', name="Dense_2"),
#     keras.layers.Dropout(0.5),
#     keras.layers.Dense(units = 800, activation='relu', name="Dense_3"),
#     keras.layers.Dense(units = 800, activation='relu', name="Dense_4"),
#     keras.layers.Dropout(0.5),
#     keras.layers.Dense(units = 200, activation='relu', name="Dense_5"),
    keras.layers.Dense(units = 200, activation='relu', name="Dense_6"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units = 37, activation='softmax', name="Dense_FINAL")
])

In [7]:
model.compile(optimizer="nadam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],)

In [8]:
letters = tf.constant(np.array(letters, dtype="float32").reshape(max_val - 1, 80, 60, 1))
keys = tf.constant(np.array(keys, dtype="float32").reshape(max_val - 1, 1))

print(letters.shape)
print(keys.shape)

data = tf.data.Dataset.from_tensor_slices((letters, keys))

(3054, 80, 60, 1)
(3054, 1)


In [11]:
model.summary()
model.fit(letters, keys, batch_size=128, epochs=12)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
average_pooling2d (AveragePo (None, 40, 30, 1)         0         
_________________________________________________________________
First_Conv2D (Conv2D)        (None, 40, 30, 64)        10880     
_________________________________________________________________
average_pooling2d_1 (Average (None, 20, 15, 64)        0         
_________________________________________________________________
Second_Conv2D (Conv2D)       (None, 20, 15, 128)       663680    
_________________________________________________________________
average_pooling2d_2 (Average (None, 10, 7, 128)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8960)              0         
_________________________________________________________________
Dense_6 (Dense)              (None, 200)               1

In [12]:
number_array = np.zeros((1, 80, 60, 1), dtype="float32")
number_array[0] = letters[2]
history = model.predict(tf.constant(np.array(number_array, dtype="float32")))
print(return_letter_dict[np.argmax(history)])

6


In [13]:
model._name = "License_Plate_Model"

In [14]:
model.save("/Users/tristanbrigham/GithubProjects/AzimuthInternship/MrCrutcherProjects/LicensePlateProject/model.h5")

In [15]:
model2 = tf.keras.models.load_model("/Users/tristanbrigham/GithubProjects/AzimuthInternship/MrCrutcherProjects/LicensePlateProject/model.h5")

In [16]:
history = model2.predict(tf.constant(np.array(number_array, dtype="float32")))
print(return_letter_dict[np.argmax(history)])

6


In [17]:
model.summary()

Model: "License_Plate_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
average_pooling2d (AveragePo (None, 40, 30, 1)         0         
_________________________________________________________________
First_Conv2D (Conv2D)        (None, 40, 30, 64)        10880     
_________________________________________________________________
average_pooling2d_1 (Average (None, 20, 15, 64)        0         
_________________________________________________________________
Second_Conv2D (Conv2D)       (None, 20, 15, 128)       663680    
_________________________________________________________________
average_pooling2d_2 (Average (None, 10, 7, 128)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8960)              0         
_________________________________________________________________
Dense_6 (Dense)              (None, 200)       

In [18]:
model2.summary()

Model: "License_Plate_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
average_pooling2d (AveragePo (None, 40, 30, 1)         0         
_________________________________________________________________
First_Conv2D (Conv2D)        (None, 40, 30, 64)        10880     
_________________________________________________________________
average_pooling2d_1 (Average (None, 20, 15, 64)        0         
_________________________________________________________________
Second_Conv2D (Conv2D)       (None, 20, 15, 128)       663680    
_________________________________________________________________
average_pooling2d_2 (Average (None, 10, 7, 128)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8960)              0         
_________________________________________________________________
Dense_6 (Dense)              (None, 200)       